#**Lending Club: An EDA Case Study**


This notebook explores loan default prediction using Exploratory Data Analysis (EDA). We'll analyze data from a consumer finance company to identify patterns indicative of loan defaults.

By examining past loan applications, our goal is to uncover crucial consumer and loan attributes that influence repayment tendencies. This insight will assist the company in refining its risk assessment strategies and reducing credit losses.

Here, we've utilized risk analytics concepts, particularly focusing on variable types and their impact on defaults.

Let's commence by uncovering the hidden patterns within the loan data and constructing a robust system to predict loan defaults!

In [ ]:
# Import libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Import libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Mount Google Drive (optional, for Colab notebooks)
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
# Define the file path to the loan data
file_path = '/content/drive/MyDrive/lending-club-case-study/loan.csv'
#file_path = '/loan.csv'

# Read the loan data into a pandas DataFrame
df = pd.read_csv(file_path)

# Filter the DataFrame to exclude loans with 'Current' status
# We'll focus on 'Fully Paid' and 'Charged Off' (defaulted) loans for analysis
df = df[df['loan_status'] != 'Current']

# Drop duplicates based on 'id' and 'member_id'
df_unique = df.drop_duplicates(subset=['id', 'member_id'], keep='first')  # Keep the first occurrence

# Display the first few rows of the filtered DataFrame (head())
print(df.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501    1296599       5000         5000           4975.0   36 months   
1  1077430    1314167       2500         2500           2500.0   60 months   
2  1077175    1313524       2400         2400           2400.0   36 months   
3  1076863    1277178      10000        10000          10000.0   36 months   
5  1075269    1311441       5000         5000           5000.0   36 months   

  int_rate  installment grade sub_grade             emp_title emp_length  \
0   10.65%       162.87     B        B2                   NaN  10+ years   
1   15.27%        59.83     C        C4                 Ryder   < 1 year   
2   15.96%        84.33     C        C5                   NaN  10+ years   
3   13.49%       339.31     C        C1   AIR RESOURCES BOARD  10+ years   
5    7.90%       156.46     A        A4  Veolia Transportaton    3 years   

  home_ownership  annual_inc verification_status issue_d  loan_status  \
0

In [32]:
# Generate summary statistics of the DataFrame
print(df.describe())

                 id     member_id     loan_amnt   funded_amnt  \
count  3.857700e+04  3.857700e+04  38577.000000  38577.000000   
mean   6.763787e+05  8.422843e+05  11047.025430  10784.058506   
std    2.092639e+05  2.644519e+05   7348.441646   7090.306027   
min    5.473400e+04  7.069900e+04    500.000000    500.000000   
25%    5.120330e+05  6.611310e+05   5300.000000   5200.000000   
50%    6.564230e+05  8.392920e+05   9600.000000   9550.000000   
75%    8.291460e+05  1.037336e+06  15000.000000  15000.000000   
max    1.077501e+06  1.314167e+06  35000.000000  35000.000000   

       funded_amnt_inv   installment    annual_inc           dti  \
count     38577.000000  38577.000000  3.857700e+04  38577.000000   
mean      10222.481123    322.466318  6.877797e+04     13.272727   
std        7022.720644    208.639215  6.421868e+04      6.673044   
min           0.000000     15.690000  4.000000e+03      0.000000   
25%        5000.000000    165.740000  4.000000e+04      8.130000   
50%   

In [33]:
# Display the column names of the DataFrame
print(df.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)


In [34]:
# Increase the maximum number of columns and rows displayed in the output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Calculate percentage of missing values in each column
cols_null_val_perctg = df.isnull().mean() * 100

# Display the percentage of missing values in each column
print(cols_null_val_perctg)

id                                  0.000000
member_id                           0.000000
loan_amnt                           0.000000
funded_amnt                         0.000000
funded_amnt_inv                     0.000000
term                                0.000000
int_rate                            0.000000
installment                         0.000000
grade                               0.000000
sub_grade                           0.000000
emp_title                           6.185033
emp_length                          2.677761
home_ownership                      0.000000
annual_inc                          0.000000
verification_status                 0.000000
issue_d                             0.000000
loan_status                         0.000000
pymnt_plan                          0.000000
url                                 0.000000
desc                               32.477901
purpose                             0.000000
title                               0.028514
zip_code  

In [35]:
# Define a threshold for acceptable missing value percentage
threshold = 30

# Filter columns with missing value percentage below the threshold
cols_to_keep = cols_null_val_perctg[cols_null_val_perctg < threshold].index

# Display the columns to be kept
print(cols_to_keep)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')


In [36]:
# Reduce DataFrame to columns with acceptable missing value percentage
df = df[cols_to_keep]

# Identify columns with non-negative unique value count
non_neg_cols = df.nunique()  # Count unique values in each column
non_neg_cols = non_neg_cols[non_neg_cols > 1].index  # Filter for columns with > 1 unique value

# Reduce DataFrame to columns with more than 1 unique value
df = df[non_neg_cols]

# Display information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38577 entries, 0 to 39716
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       38577 non-null  int64  
 1   member_id                38577 non-null  int64  
 2   loan_amnt                38577 non-null  int64  
 3   funded_amnt              38577 non-null  int64  
 4   funded_amnt_inv          38577 non-null  float64
 5   term                     38577 non-null  object 
 6   int_rate                 38577 non-null  object 
 7   installment              38577 non-null  float64
 8   grade                    38577 non-null  object 
 9   sub_grade                38577 non-null  object 
 10  emp_title                36191 non-null  object 
 11  emp_length               37544 non-null  object 
 12  home_ownership           38577 non-null  object 
 13  annual_inc               38577 non-null  float64
 14  verification_status      38

In [37]:
# Create a new column for Debt-to-Income Ratio (monthly)
df['Debt/Income Ratio'] = df['installment'] * 100 / (df['annual_inc'] / 12)

In [38]:
# Define lists to categorize columns by data type or characteristic

num_cols = [  # Numerical columns
    'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment',
    'annual_inc', 'dti', 'revol_bal', 'revol_util', 'total_pymnt',
    'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'Debt/Income Ratio'
]

cat_cols = [  # Categorical columns
    'term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
    'verification_status', 'loan_status', 'purpose', 'addr_state', 'delinq_2yrs',
    'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'pub_rec_bankruptcies'
]

date_cols = [  # Date columns
    'issue_d', 'last_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line'
]

extra_cols = [  # Columns to potentially exclude from analysis
    'id', 'member_id', 'emp_title', 'pymnt_plan', 'url', 'title', 'zip_code'
]

# Calculate the total number of columns across all categories
total_cols = len(num_cols) + len(cat_cols) + len(extra_cols) + len(date_cols)

print(total_cols)  # Print the total number of columns

44


In [39]:
# Get summary information for numerical columns

print(df[num_cols].info())

<class 'pandas.core.frame.DataFrame'>
Index: 38577 entries, 0 to 39716
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                38577 non-null  int64  
 1   funded_amnt              38577 non-null  int64  
 2   funded_amnt_inv          38577 non-null  float64
 3   int_rate                 38577 non-null  object 
 4   installment              38577 non-null  float64
 5   annual_inc               38577 non-null  float64
 6   dti                      38577 non-null  float64
 7   revol_bal                38577 non-null  int64  
 8   revol_util               38527 non-null  object 
 9   total_pymnt              38577 non-null  float64
 10  total_pymnt_inv          38577 non-null  float64
 11  total_rec_prncp          38577 non-null  float64
 12  total_rec_int            38577 non-null  float64
 13  total_rec_late_fee       38577 non-null  float64
 14  recoveries               38

In [40]:
# Calculate missing value percentage for each numerical column
print(df[num_cols].isnull().mean() * 100)

loan_amnt                  0.000000
funded_amnt                0.000000
funded_amnt_inv            0.000000
int_rate                   0.000000
installment                0.000000
annual_inc                 0.000000
dti                        0.000000
revol_bal                  0.000000
revol_util                 0.129611
total_pymnt                0.000000
total_pymnt_inv            0.000000
total_rec_prncp            0.000000
total_rec_int              0.000000
total_rec_late_fee         0.000000
recoveries                 0.000000
collection_recovery_fee    0.000000
last_pymnt_amnt            0.000000
Debt/Income Ratio          0.000000
dtype: float64


In [ ]:
# Preprocess 'revol_util' column as it has % sign in the values

# Convert percentages to numeric
try:
  df['revol_util'] = pd.to_numeric(df['revol_util'].str.replace('%', ''), errors='coerce')
except:
  # If conversion fails, replace with NaN
  df['revol_util'] = pd.to_numeric(df['revol_util'].str.replace('%', ''), errors='replace')

# Convert from percentage to decimal (divide by 100)
df['revol_util'] = df['revol_util'] / 100

# Check for missing values before and after filling

# Print number of NaN values before filling
print("Number of NaN values before fillna:", df['revol_util'].isna().sum())

# Fill NaN values with mean (assuming missing values are a small proportion)
mean_revolve_util = df['revol_util'].median()
df['revol_util'].fillna(mean_revolve_util, inplace=True)

# Print number of NaN values after filling
print("Number of NaN values after fillna:", df['revol_util'].isna().sum())

Number of NaN values before fillna: 4
Number of NaN values after fillna: 0


In [ ]:
# Calculate missing value percentage for each categorical column
print(df[cat_cols].isnull().mean() * 100)

term                    0.000000
grade                   0.000000
sub_grade               0.011596
emp_length              3.594620
home_ownership          0.011596
verification_status     0.011596
loan_status             0.011596
purpose                 0.011596
addr_state              0.011596
delinq_2yrs             0.011596
inq_last_6mths          0.011596
open_acc                0.011596
pub_rec                 0.011596
total_acc               0.011596
pub_rec_bankruptcies    0.011596
dtype: float64


In [ ]:
# Preprocess 'emp_length' column (as it contains 'years' and 'year' suffix)

# Remove potential suffixes ('years' or 'year') using regular expressions
df['emp_length'] = df['emp_length'].str.replace('years?', '', regex=True)

# Fill missing values with the most frequent value (mode)
mode_emp_length = df['emp_length'].mode()
print("Number of NaN values before fillna:", df['emp_length'].isna().sum())
df['emp_length'].fillna(mode_emp_length[0], inplace=True)
print("Number of NaN values after fillna:", df['emp_length'].isna().sum())

In [ ]:
# Filling missing values in 'pub_rec_bankruptcies' with the mode (most frequent value)

mode_emp_length = df['pub_rec_bankruptcies'].mode()

print("Number of NaN values before fillna:", df['pub_rec_bankruptcies'].isna().sum())
df['pub_rec_bankruptcies'].fillna(mode_emp_length[0], inplace=True)
print("Number of NaN values after fillna:", df['pub_rec_bankruptcies'].isna().sum())

Number of NaN values before fillna: 1
Number of NaN values after fillna: 0


In [ ]:
# Univariate Analysis for Numerical Columns using Seaborn

for num in num_cols:
  print('\n Creating Histogram for:', num)
  plt.figure(figsize=(16, 5))  # Adjusting figure size for aesthetics

  # Customizing plot appearance using Matplotlib
  sns.histplot(
      df[num],
      color="#3f5d7d",  # Set bar color
      edgecolor="white",  # Set bar edge color
      kde=True,  # Add a density plot line
      linewidth=0.8,  # Adjust line width
      bins=20  # Adjust number of bins for smoother distribution visualization
  )

  # Customize labels and title using Matplotlib
  plt.xlabel(num, labelpad=10)  # Set x-axis label with padding
  plt.xticks(rotation=60)
  plt.ylabel('Frequency', labelpad=10)  # Set y-axis label with padding
  plt.title(f'Distribution of {num}', fontsize=14)  # Set informative title
  plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

  plt.tight_layout()  # Adjust spacing for readability
  plt.show()


In [ ]:
# Univariate Analysis for Numerical Columns with Styled Boxplots
for num in num_cols:
    print('\nCreating Boxplot for:', num)
    plt.figure(figsize=(8, 10))  # Adjust figure size for boxplots

    # Customize plot appearance using Seaborn
    sns.boxplot(
        y=num,  # Set variable on y-axis
        data=df,  # Set data frame
        showmeans=True,  # Display means as markers
        color="#ff9999"  # Set box color
    )

    # Customize labels and title using Matplotlib
    plt.xlabel('Value', labelpad=10)  # Set x-axis label with padding
    plt.ylabel(num, labelpad=10)  # Set y-axis label with padding
    plt.title(f'Distribution of {num}', fontsize=14)  # Set informative title
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

    plt.tight_layout()  # Adjust spacing for readability
    plt.show()


In [ ]:
# Categorical Columns Analysis using Countplots

for cat in cat_cols:
    print('\nCreating Countplot for:', cat)
    plt.figure(figsize=(16, 6))  # Adjust figure size for aesthetics

    # Create countplot with customizations using seaborn
    sns.countplot(
        x=cat,
        data=df,  # Use df directly as the data parameter
        palette="Set3",  # Set color palette
        edgecolor=".7",  # Set bar edge color
    )

    # Customize labels and title using Matplotlib
    plt.xlabel(cat, labelpad=10)  # Set x-axis label with padding
    plt.ylabel('Count', labelpad=10)  # Set y-axis label with padding
    plt.title(f'Distribution of {cat}', fontsize=14)  # Set informative title
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability (optional)
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

    plt.tight_layout()  # Adjust spacing for readability
    plt.show()

In [ ]:
# Bivariate Analysis for Numerical Columns with Styled Scatterplots

for i in range(len(num_cols)):
    for j in range(i+1, len(num_cols)):  # Consider only upper triangular matrix to avoid duplicate plots
        num = num_cols[i]
        num_var = num_cols[j]

        print(f'Creating Scatterplot for: {num} vs {num_var}')
        plt.figure(figsize=(25, 6))  # Adjust figure size

        # Create scatterplot with customizations using seaborn
        sns.scatterplot(
            x=num,
            y=num_var,
            data=df,
            alpha=0.7,  # Adjust transparency
            linewidth=0.8,  # Adjust line width for data points
            palette="Set2",  # Set color palette
            edgecolor=".2",  # Set data point edge color
        )

        # Customize labels and title using Matplotlib
        plt.xlabel(num, labelpad=10)  # Set x-axis label with padding
        plt.ylabel(num_var, labelpad=10)  # Set y-axis label with padding
        plt.title(f'Scatterplot: {num} vs {num_var}', fontsize=14)  # Set informative title
        plt.grid(True, linestyle='--', alpha=0.6)  # Adjust gridlines for readability

        plt.tight_layout()  # Adjust spacing for readability
        plt.show()


In [ ]:
# Categorical vs Loan Status Analysis with Styled Boxplots

for cat in cat_cols:
  print(f"\nCreating Boxplot for: {cat} vs Loan Status")
  plt.figure(figsize=(8, 10))  # Adjust figure size for readability

  # Create boxplot using seaborn with customizations
  sns.boxplot(
      x="loan_status",
      y=cat,
      showmeans=True,  # Display means as markers
      palette="Set3",  # Set color palette
      data=df
  )

  # Customize labels and title using Matplotlib
  plt.xlabel("Loan Status", labelpad=10)  # Set x-axis label with padding
  plt.ylabel(cat, labelpad=10)  # Set y-axis label with padding
  plt.title(f'Distribution of {cat} by Loan Status', fontsize=14)  # Set informative title
  plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
  plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

  # Print value counts for informative purposes (optional)
  print(df[cat].value_counts())

  plt.tight_layout()  # Adjust spacing for readability
  plt.show()


In [ ]:
# Numerical vs Loan Status Analysis with Styled Boxplots

for num in num_cols:
  print(f"\nCreating Boxplot for: {num} vs Loan Status")
  plt.figure(figsize=(8, 10))  # Adjust figure size for readability

  # Create boxplot using seaborn with customizations
  sns.boxplot(
      x="loan_status",
      y=num,
      showmeans=True,  # Display means as markers
      palette="Set3",  # Set color palette
      data=df
  )

  # Customize labels and title using Matplotlib
  plt.xlabel("Loan Status", labelpad=10)  # Set x-axis label with padding
  plt.ylabel(num, labelpad=10)  # Set y-axis label with padding
  plt.title(f'Distribution of {num} by Loan Status', fontsize=14)  # Set informative title
  plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
  plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

  plt.tight_layout()  # Adjust spacing for readability
  plt.show()


In [ ]:
# Multivariate Analysis: Correlation Heatmap

# Ensure 'int_rate' is numerical (assuming it's a percentage)
df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)

# Correlation matrix calculation (consider using abs() for stronger visuals)
corr_matrix = df[num_cols].corr()

# Create correlation heatmap using seaborn with customizations
plt.figure(figsize=(12, 8))  # Adjust figure size for readability
sns.heatmap(
    corr_matrix,
    annot=True,  # Display correlation values within heatmap cells
    cmap="YlGnBu",  # Set colormap (consider diverging colormap like "coolwarm")
    vmin=-1,  # Set minimum value for color range (-1 for stronger visuals)
    vmax=1  # Set maximum value for color range (1 for stronger visuals)
)

# Customize labels and title using Matplotlib
plt.xlabel('Numerical Features', labelpad=10)  # Set x-axis label with padding
plt.ylabel('Numerical Features', labelpad=10)  # Set y-axis label with padding (consider rotating for long labels)
plt.title('Correlation Heatmap of Numerical Features', fontsize=14)  # Set informative title

plt.tight_layout()  # Adjust spacing for readability
plt.show()


In [ ]:
# Grouped Bar Charts: Loan Status vs Categorical Variables

for cat_var in cat_cols:
    # Count the frequency of each combination of categories
    cross_tab = pd.crosstab(df['loan_status'], df[cat_var])

    # Create grouped bar chart with customizations
    plt.figure(figsize=(12, 8))  # Adjust figure size for readability
    ax = cross_tab.plot(kind='bar', colormap='Set3')  # Set color palette

    # Customize labels and title using Matplotlib
    ax.set_xlabel(cat_var, labelpad=10)  # Set x-axis label with padding
    ax.set_ylabel('Frequency', labelpad=10)  # Set y-axis label with padding
    ax.set_title(f'Distribution of {cat_var} by Loan Status', fontsize=14)  # Set informative title
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')  # Rotate x-axis labels for readability
    ax.legend(title='Loan Status', loc='upper left', bbox_to_anchor=(1, 1))  # Adjust legend position
    ax.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

    plt.tight_layout()  # Adjust spacing for readability
    plt.show()



In [ ]:
# Violin Charts: Loan Status vs Categorical Variables

# Extract numeric values from 'term' column and convert them to integers
df['term_numeric'] = df['term'].str.extract('(\d+)').astype(int)

# Set seaborn style for all plots
sns.set_style("whitegrid")

# Set color palette for consistent visuals (optional)
palette = sns.color_palette("Set2")  # Replace with desired palette

for cat_var in cat_cols:
    print(f"\nCreating Violin Plot for Loan Status vs {cat_var}")
    plt.figure(figsize=(12, 6))

    # Create violin plot using seaborn with customizations
    ax = sns.violinplot(
        x="loan_status",
        y=cat_var,
        palette=palette,
        data=df
    )

    # Customize labels and title using Matplotlib
    plt.xlabel("Loan Status", labelpad=10)  # Set x-axis label with padding
    plt.ylabel(cat_var, labelpad=10)  # Set y-axis label with padding
    plt.title(f'Distribution of {cat_var} by Loan Status', fontsize=14)  # Set informative title
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add subtle gridlines on y-axis

    plt.tight_layout()  # Adjust spacing for readability
    plt.show()